# Praktikum 3

In [ ]:
!pip install hnswlib

In [ ]:
import hnswlib
import numpy as np
import time
from sklearn.neighbors import NearestNeighbors

# ===========================
# 1. Buat data 2D acak
# ===========================
num_elements = 1000
dim = 2
data = np.random.random((num_elements, dim)).astype(np.float32)

# Query point
query = np.array([[0.5, 0.5]], dtype=np.float32)
k = 5  # cari 5 tetangga terdekat

# ===========================
# 2. Exact NN (Brute Force)
# ===========================
nn = NearestNeighbors(n_neighbors=k, algorithm='brute', metric='euclidean')
nn.fit(data)

start = time.time()
distances, indices = nn.kneighbors(query)
end = time.time()

print("=== Exact NN ===")
print("Indices:", indices)
print("Distances:", distances)
print("Waktu:", end - start, "detik")

# ===========================
# 3. HNSW
# ===========================
# Inisialisasi index HNSW
p = hnswlib.Index(space='l2', dim=dim)

# Ukuran maksimum elemen yang bisa ditampung
p.init_index(max_elements=num_elements, ef_construction=100, M=16)

# Tambahkan data
p.add_items(data)

# Set parameter pencarian
p.set_ef(50)   # tradeoff speed vs accuracy

start = time.time()
labels, distances = p.knn_query(query, k=k)
end = time.time()

print("\n=== HNSW ===")
print("Indices:", labels)
print("Distances:", distances)
print("Waktu:", end - start, "detik")


=== Exact NN ===
Indices: [[830 247 473 913 592]]
Distances: [[0.00993625 0.01363944 0.01646171 0.03779694 0.03859071]]
Waktu: 0.06820344924926758 detik

=== HNSW ===
Indices: [[830 247 473 913 592]]
Distances: [[9.8728990e-05 1.8603441e-04 2.7098786e-04 1.4286089e-03 1.4892431e-03]]
Waktu: 0.00023245811462402344 detik


In [ ]:
import hnswlib
import numpy as np
import time
import pandas as pd
from sklearn.neighbors import NearestNeighbors

# ===========================
# Fungsi untuk menjalankan satu percobaan
# ===========================
def run_hnsw_experiment(n_points=1000, dim=2, metric='l2', k=5):
    np.random.seed(42)
    data = np.random.random((n_points, dim)).astype(np.float32)
    query = np.random.random((1, dim)).astype(np.float32)

    # === Exact NN (Brute Force) ===
    nn = NearestNeighbors(n_neighbors=k, algorithm='brute', metric='euclidean' if metric == 'l2' else 'cosine')
    nn.fit(data)

    start = time.time()
    distances_exact, indices_exact = nn.kneighbors(query)
    time_exact = (time.time() - start) * 1000  # ms

    # === HNSW ===
    p = hnswlib.Index(space=metric, dim=dim)
    p.init_index(max_elements=n_points, ef_construction=100, M=16)
    p.add_items(data)
    p.set_ef(50)

    start = time.time()
    labels, distances_hnsw = p.knn_query(query, k=k)
    time_hnsw = (time.time() - start) * 1000  # ms

    # Hitung akurasi (berapa hasil sama)
    accuracy = len(set(indices_exact[0]) & set(labels[0])) / k * 100

    return {
        'Metric Distance': metric,
        'Dimensi Data': dim,
        'Jumlah Data': n_points,
        'Metode': 'Exact (Brute Force)',
        'Waktu Komputasi (ms)': round(time_exact, 4),
        'Akurasi HNSW (%)': 100.0
    }, {
        'Metric Distance': metric,
        'Dimensi Data': dim,
        'Jumlah Data': n_points,
        'Metode': 'HNSW',
        'Waktu Komputasi (ms)': round(time_hnsw, 4),
        'Akurasi HNSW (%)': round(accuracy, 2)
    }

# ===========================
# Jalankan percobaan
# ===========================
results = []
for metric in ['l2', 'cosine']:
    for n_points in [1000, 1000000]:
        for dim in [2, 5]:
            exact_res, hnsw_res = run_hnsw_experiment(n_points, dim, metric)
            results.extend([exact_res, hnsw_res])
            print(f"Selesai: metric={metric}, data={n_points}, dim={dim}")

# ===========================
# Buat tabel hasil
# ===========================
df = pd.DataFrame(results, columns=[
    'Metric Distance', 'Dimensi Data', 'Jumlah Data',
    'Metode', 'Waktu Komputasi (ms)', 'Akurasi HNSW (%)'
])

print("\n=== Hasil Percobaan HNSW ===")
print(df.to_string(index=False))


Selesai: metric=l2, data=1000, dim=2
Selesai: metric=l2, data=1000, dim=5
Selesai: metric=l2, data=1000000, dim=2
Selesai: metric=l2, data=1000000, dim=5
Selesai: metric=cosine, data=1000, dim=2
Selesai: metric=cosine, data=1000, dim=5
Selesai: metric=cosine, data=1000000, dim=2
Selesai: metric=cosine, data=1000000, dim=5

=== Hasil Percobaan HNSW ===
Metric Distance  Dimensi Data  Jumlah Data              Metode  Waktu Komputasi (ms)  Akurasi HNSW (%)
             l2             2         1000 Exact (Brute Force)                2.9502             100.0
             l2             2         1000                HNSW                0.1841             100.0
             l2             5         1000 Exact (Brute Force)                0.5980             100.0
             l2             5         1000                HNSW                0.0658             100.0
             l2             2      1000000 Exact (Brute Force)               25.0452             100.0
             l2             